In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
import json
from numpy import random
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import StackingClassifier

In [2]:
ext=('.json')
attackerlist=[]
for files in os.listdir():
    if files.__contains__('A16'):
        attackerlist.append(int(files.split("-")[2]))
print(attackerlist)

[523, 643, 145, 373, 571, 481, 109, 97, 625, 331, 349, 319, 289, 511, 91, 589, 115, 229, 67, 175, 241, 265, 421, 73]


In [3]:
if(os.path.isfile('bsm.csv')):
    os.remove('bsm.csv')
for files in os.listdir():
    if files.endswith(ext):
        atk = pd.read_json(files,lines=True)
        atk.drop(['noise','spd_noise','pos_noise'], inplace=True, axis=1,errors='ignore')
        atk['reciever']=int(files.split("-")[2])
        atk1 = pd.DataFrame(atk['pos'].tolist(), columns=['pos-x1', 'pos-y1', 'pos-z'])
        atk1.drop(['pos-z'], inplace=True, axis=1,errors='ignore')
        atk2 = pd.DataFrame(atk['spd'].tolist(), columns=['spd-x1', 'spd-y1', 'spd-z'])
        atk2.drop(['spd-z'], inplace=True, axis=1,errors='ignore')
        frames1=[atk,atk1,atk2]
        attack=pd.concat(frames1,axis=1)
        attack.drop(['pos','spd'], inplace=True, axis=1,errors='ignore')
        bsm = attack[attack["type"] == 3]
        cs = ['pos-x1','pos-y1','spd-x1','spd-y1']
        g= bsm[cs].shift(-1).add_prefix('prev')
        bsm = pd.concat([bsm, g], axis=1).fillna(0).astype(float)
        bsm['attacker']=0
        if 'sender' in bsm:
            bsm.loc[bsm["sender"].isin(attackerlist), "attacker"] = 1
        else:
            continue
        bsm.drop(['type','rcvTime','sendTime','messageID'], inplace=True, axis=1,errors='ignore')
        bsm.to_csv('bsm.csv', mode='a', header=False, index=False)

In [4]:
col_names = ['sender','RSSI','reciever','sen-px','sen-py','sen-sx','sen-sy','prev-px','prev-py','prev-sx','prev-sy','attacker']
# load dataset
pi = pd.read_csv("bsm.csv", header=None, names=col_names)

pi['del-px'] = pi['sen-px'] - pi['prev-px']
pi['del-py'] = pi['sen-py'] - pi['prev-py']
pi['del-sx'] = pi['sen-sx'] - pi['prev-sx']
pi['del-sy'] = pi['sen-sy'] - pi['prev-sy']

pi["receiver-px"]= np.random.permutation(pi["sen-px"].values)
pi["receiver-py"]= np.random.permutation(pi["sen-py"].values)
pi["receiver-sx"]= np.random.permutation(pi["sen-sx"].values)
pi["receiver-sy"]= np.random.permutation(pi["sen-sy"].values)

cs = ['receiver-px','receiver-py'	,'receiver-sx'	,'receiver-sy']
g= pi[cs].shift(-1).add_prefix('prev-')
pi = pd.concat([pi, g], axis=1).fillna(0).astype(float)


pi['AoA'] = np.arctan((pi['receiver-py']-pi['sen-py'])/(pi['receiver-px']-pi['sen-px']))
pi['delr-px'] = pi['receiver-px'] - pi['prev-receiver-px']
pi['delr-py'] = pi['receiver-py'] - pi['prev-receiver-py']
pi['delr-sx'] = pi['receiver-sx'] - pi['prev-receiver-sx']
pi['delr-sy'] = pi['receiver-sy'] - pi['prev-receiver-sy']


ctm = pi.pop('attacker');
pi.insert(len(pi.columns),'attacker',ctm)
pi

,sender,RSSI,reciever,sen-px,sen-py,sen-sx,sen-sy,prev-px,prev-py,prev-sx,...,prev-receiver-px,prev-receiver-py,prev-receiver-sx,prev-receiver-sy,AoA,delr-px,delr-py,delr-sx,delr-sy,attacker
0,463.0,9.711834e-08,523.0,3591.065707,5213.340545,13.638606,1.577790,3598.068679,5662.706996,1.616905,...,3602.601968,6021.572992,-4.266614e-01,-31.729023,1.539341,14.949331,33.499255,3.085102,29.580660,0.0
1,451.0,1.590352e-09,523.0,3598.068679,5662.706996,1.616905,-37.651837,3611.927158,5367.364995,-3.992088,...,5781.654929,5192.885583,0.000000e+00,-0.427933,1.558165,-2179.052962,828.687409,-0.426661,-31.301090,0.0
2,505.0,2.796052e-09,523.0,3611.927158,5367.364995,-3.992088,34.605169,3598.807422,6013.174387,-0.020641,...,5914.589888,5749.734532,-1.269356e+01,0.243497,-0.080243,-132.934959,-556.848948,12.693565,-0.671431,0.0
3,481.0,2.693857e-09,523.0,3598.807422,6013.174387,-0.020641,-35.463332,3684.213239,5230.399464,13.548951,...,3840.144794,5474.520687,2.322297e-15,-34.876827,-0.113272,2074.445094,275.213844,-12.693565,35.120324,1.0
4,445.0,2.352880e-08,523.0,3684.213239,5230.399464,13.548951,3.963743,3597.799697,5814.875922,-0.019687,...,6126.265693,5515.926896,-1.406847e+01,35.483894,1.002373,-2286.120899,-41.406209,14.068470,-70.360721,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25076,367.0,7.712088e-07,313.0,6138.178602,5737.603466,12.356516,0.632416,6147.642308,5823.870461,-2.616917,...,2484.354085,5588.682559,2.221304e+00,33.004608,-0.000765,1135.401654,150.847263,-2.221304,-31.893280,0.0
25077,643.0,1.314467e-08,313.0,6147.642308,5823.870461,-2.616917,-0.263639,5874.199580,5712.895935,6.153583,...,3587.151279,5985.034273,0.000000e+00,30.750336,0.064113,-1102.797193,-396.351714,2.221304,2.254273,1.0
25078,403.0,4.088686e-09,313.0,5874.199580,5712.895935,6.153583,0.709647,4460.814666,6017.115503,11.823865,...,3638.328919,5741.978633,0.000000e+00,1.876175,-0.118434,-51.177640,243.055640,0.000000,28.874161,0.0
25079,229.0,9.602766e-08,313.0,4460.814666,6017.115503,11.823865,3.192250,6324.882635,5732.777983,-13.696731,...,3642.574531,5985.034273,-1.189259e+01,-1.067884,0.322817,-4.245613,-243.055640,11.892593,2.944058,1.0


In [5]:
assert isinstance(pi, pd.DataFrame), "df needs to be a pd.DataFrame"
pi.dropna(inplace=True)
indices_to_keep = ~pi.isin([np.nan, np.inf, -np.inf]).any(1)
pi=pi[indices_to_keep].astype(np.float64)
pi = pi.astype({"attacker": int})
pi

,sender,RSSI,reciever,sen-px,sen-py,sen-sx,sen-sy,prev-px,prev-py,prev-sx,...,prev-receiver-px,prev-receiver-py,prev-receiver-sx,prev-receiver-sy,AoA,delr-px,delr-py,delr-sx,delr-sy,attacker
0,463.0,9.711834e-08,523.0,3591.065707,5213.340545,13.638606,1.577790,3598.068679,5662.706996,1.616905,...,3602.601968,6021.572992,-4.266614e-01,-31.729023,1.539341,14.949331,33.499255,3.085102,29.580660,0
1,451.0,1.590352e-09,523.0,3598.068679,5662.706996,1.616905,-37.651837,3611.927158,5367.364995,-3.992088,...,5781.654929,5192.885583,0.000000e+00,-0.427933,1.558165,-2179.052962,828.687409,-0.426661,-31.301090,0
2,505.0,2.796052e-09,523.0,3611.927158,5367.364995,-3.992088,34.605169,3598.807422,6013.174387,-0.020641,...,5914.589888,5749.734532,-1.269356e+01,0.243497,-0.080243,-132.934959,-556.848948,12.693565,-0.671431,0
3,481.0,2.693857e-09,523.0,3598.807422,6013.174387,-0.020641,-35.463332,3684.213239,5230.399464,13.548951,...,3840.144794,5474.520687,2.322297e-15,-34.876827,-0.113272,2074.445094,275.213844,-12.693565,35.120324,1
4,445.0,2.352880e-08,523.0,3684.213239,5230.399464,13.548951,3.963743,3597.799697,5814.875922,-0.019687,...,6126.265693,5515.926896,-1.406847e+01,35.483894,1.002373,-2286.120899,-41.406209,14.068470,-70.360721,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25076,367.0,7.712088e-07,313.0,6138.178602,5737.603466,12.356516,0.632416,6147.642308,5823.870461,-2.616917,...,2484.354085,5588.682559,2.221304e+00,33.004608,-0.000765,1135.401654,150.847263,-2.221304,-31.893280,0
25077,643.0,1.314467e-08,313.0,6147.642308,5823.870461,-2.616917,-0.263639,5874.199580,5712.895935,6.153583,...,3587.151279,5985.034273,0.000000e+00,30.750336,0.064113,-1102.797193,-396.351714,2.221304,2.254273,1
25078,403.0,4.088686e-09,313.0,5874.199580,5712.895935,6.153583,0.709647,4460.814666,6017.115503,11.823865,...,3638.328919,5741.978633,0.000000e+00,1.876175,-0.118434,-51.177640,243.055640,0.000000,28.874161,0
25079,229.0,9.602766e-08,313.0,4460.814666,6017.115503,11.823865,3.192250,6324.882635,5732.777983,-13.696731,...,3642.574531,5985.034273,-1.189259e+01,-1.067884,0.322817,-4.245613,-243.055640,11.892593,2.944058,1


In [6]:
X = pi.iloc[:,:-1].values
y = pi.iloc[:, 28].values


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.99)

scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)

knn_y_pred = knn.predict(X_test)

print(confusion_matrix(y_test, knn_y_pred))
print(classification_report(y_test, knn_y_pred))

[[17887  1050]
 [ 3854  2039]]
              precision    recall  f1-score   support

           0       0.82      0.94      0.88     18937
           1       0.66      0.35      0.45      5893

    accuracy                           0.80     24830
   macro avg       0.74      0.65      0.67     24830
weighted avg       0.78      0.80      0.78     24830



/Users/adithyabalaji/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:237: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


In [7]:
rf = RandomForestClassifier(n_estimators=100)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)
rf.fit(X_train,y_train)

rf_y_pred = rf.predict(X_test)

print(confusion_matrix(y_test, rf_y_pred))
print(classification_report(y_test, rf_y_pred))

[[18316   621]
 [ 2599  3294]]
              precision    recall  f1-score   support

           0       0.88      0.97      0.92     18937
           1       0.84      0.56      0.67      5893

    accuracy                           0.87     24830
   macro avg       0.86      0.76      0.80     24830
weighted avg       0.87      0.87      0.86     24830



In [8]:
lr = LogisticRegression()
clfStack = StackingClassifier(estimators=[('knn',knn),('rf',rf)],final_estimator=lr,cv=10)

clfStack.fit(X_train,y_train);

st_pred = clfStack.predict(X_test)

print(confusion_matrix(y_test, st_pred))
print(classification_report(y_test, st_pred))


[[18594   343]
 [ 3067  2826]]
              precision    recall  f1-score   support

           0       0.86      0.98      0.92     18937
           1       0.89      0.48      0.62      5893

    accuracy                           0.86     24830
   macro avg       0.88      0.73      0.77     24830
weighted avg       0.87      0.86      0.85     24830

